In [ ]:
import numpy as np
import astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
import gammapy
from gammapy.data import DataStore
from collections import OrderedDict
print('Astropy version:', astropy.__version__)
print('Gammapy version:', gammapy.__version__)

Astropy version: 4.0.1.post1
Gammapy version: 0.17


In [2]:
max_offset = 2.0
src_radius = 0.3
config = 'std_ImPACT_fullEnclosure'

In [3]:
e = np.logspace(-2, 2, 97) * u.TeV

In [4]:
# Get run list
runs = np.loadtxt('crab_hess1_above20000.lis', usecols=(0,), dtype=int)

In [5]:
# Init DataStore
ds = DataStore.from_dir('$HESS_DATA/hess1/{}'.format(config))
obs_list = ds.get_observations(runs)

In [6]:
ra_obj = 83.6333
dec_obj = 22.0144
target = SkyCoord(ra_obj, dec_obj, frame='icrs', unit='deg')

In [7]:
thresholds = OrderedDict()
for obs in obs_list:
    # calculate offset to source position,
    # make sure max_offset is not chosen too small
    offset = obs.pointing_radec.separation(target)
    thr_offset = max_offset * u.deg
    min_offset = offset + src_radius * u.deg
    assert thr_offset >= min_offset

    # extract energy dispersion for maximum offset
    edisp = obs.edisp.to_energy_dispersion(thr_offset, e_reco=e, e_true=e)
    #print(edisp)
    # determine threshold from energy dispersion
    threshold = edisp.get_bias_energy(0.1, emax=10*u.TeV)[0]
    #print(threshold)
    # use lower edge of first bin above threshold
    threshold = e[np.where(~(e[:-1]<threshold))[0][0]]
    #print(np.where(~(e[:-1]<threshold))[0][0])
    #print(e[28])
    # round such that the threshold is always slightly *below* the bin edge
    threshold = np.floor(1e4*threshold)/1e4

    print(obs.obs_id, ' offset={:.4f}  threshold={:.4f}'.format(offset, threshold))
    thresholds[obs.obs_id] = threshold.value

23037  offset=0.5000 deg  threshold=1.9573 TeV
23063  offset=0.5000 deg  threshold=1.7782 TeV
23304  offset=0.5027 deg  threshold=1.2115 TeV
23309  offset=0.5000 deg  threshold=0.8254 TeV
23310  offset=0.5000 deg  threshold=1.0000 TeV
23523  offset=0.5000 deg  threshold=0.9085 TeV
23526  offset=0.5000 deg  threshold=0.6812 TeV
23544  offset=0.5000 deg  threshold=0.9085 TeV
23545  offset=0.4999 deg  threshold=0.7498 TeV
23546  offset=0.5000 deg  threshold=0.6812 TeV
23547  offset=0.5000 deg  threshold=0.6812 TeV
23555  offset=0.0001 deg  threshold=1.4677 TeV
23556  offset=0.5007 deg  threshold=1.1006 TeV
23557  offset=1.0013 deg  threshold=0.9085 TeV
23558  offset=0.5007 deg  threshold=0.7498 TeV
23559  offset=1.5019 deg  threshold=0.6812 TeV
23576  offset=0.0001 deg  threshold=0.7498 TeV
23577  offset=0.5006 deg  threshold=0.6812 TeV
23579  offset=0.5006 deg  threshold=0.8254 TeV
23592  offset=1.5018 deg  threshold=0.9085 TeV
23593  offset=0.4635 deg  threshold=0.7498 TeV
23594  offset

In [8]:
np.savetxt('thresholds_bias_4Tel_above20000{}.txt'.format(config),
           np.array([list(thresholds.keys()), list(thresholds.values())]).T,
           fmt=['%d', '%.4f'], header='Run | Threshold [TeV]')